In [22]:
import pandas as pd
import numpy as np

import api,utils

If it's not making sense - there's removed code at line 160 of api.py

In [2]:
competition_types = api.get_competition_types()
competition_types = competition_types[competition_types['Abbreviation'].isin(["WCH", "U23WCH", "JWCH"])]
venues = api.get_venues()['RegionCity']

def get_relevant_competitions(year:int)->pd.DataFrame:
    """ Inputs an year and returns a dataframe with ids and venues of any Regular, Junior and U23 World Championships from that year""" 

    competition_ids_df = api.get_competitions(year = year)
    competition_ids_df = competition_ids_df[competition_ids_df['competitionTypeId'].isin(competition_types.index)]
    competition_ids_df = competition_ids_df[['Year', 'venueId', 'competitionTypeId']].merge(competition_types['Abbreviation'], how= "left",left_on='competitionTypeId', right_index= True)

    competition_ids_df['City'] = competition_ids_df['venueId'].apply((lambda x: venues[x]))

    competition_ids_df = competition_ids_df[['Year', 'Abbreviation', 'City']]
    competition_ids_df.rename(columns={'Abbreviation':'CompetitionType'}, inplace=True)

    return competition_ids_df

In [3]:
competitions_df = pd.DataFrame()

for year in range(1980, 2025):
    competitions_df = pd.concat([competitions_df, get_relevant_competitions(year)])
    print(year)

display(competitions_df)

1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024


,Year,CompetitionType,City
id,,,
8695e858-ceeb-4b76-82f5-1acc24c47db5,1980,WCH,Hazewinkel
c3446ea8-92c3-4646-b47a-fe2e84bcee97,1980,JWCH,Hazewinkel
45a27735-6626-48f7-a50b-bc127e248238,1981,JWCH,Sofia
9d60027c-eaa6-46c9-ae62-ac430c1da031,1981,WCH,Munich
a9dc6674-81af-4a3c-950f-af03713e0c67,1982,JWCH,Piediluco
...,...,...,...
e425631d-7ee3-454f-b34e-b2f79b9627f2,2023,JWCH,Paris
bd86933d-79f1-4f4e-8506-fbc04835e8d5,2023,WCH,Belgrade
32228428-db12-474e-8fdd-891f280a5ee0,2024,U23WCH,St. Catharines


In [4]:
competitions_df.to_csv('competition_df.csv')

In [12]:
competitions_df=pd.read_csv('competition_df.csv', index_col='id')

In [13]:
api.get_boat_types()

,DisplayName,Order
id,,
02316e75-fdf2-4660-af40-48dda1867e1f,M2-,9
079e71bb-98cd-47f4-8ca4-e3fd0cdbc538,JM1x,100
07c32034-57cf-46cd-a4c5-2fe8dbb0aa25,LTAID4+,100
0f218eeb-6f65-48c7-941c-97f0901cb8cd,BLW2-,100
1163bebd-2c43-441f-97d5-7f2bd57fd062,M2+,100
...,...,...
eaf690e9-cc36-45bc-b59f-6d510c16daf7,CMix4x+,100
f1123970-de2b-462e-89a5-f74e0b447939,LW8+,100
f7576033-2298-4549-ad10-cdbfc78417b0,CJM1x,100


In [28]:
def get_competitions_results_REDO(competition_id=None, finals_only=False, boat_classes=None):
    competition_races = api.get_competition_races(competition_id)
    competition_events = api.get_competition_events(competition_id)
    competition_results = api.get_race_results(competition_id=competition_id)
    if boat_classes is None: boat_classes = api.get_boat_types()

    if competition_results.empty:
        return competition_results

    if finals_only:
        competition_races = competition_races.loc[
            competition_races.racePhaseId == api.RACE_PHASES["Final"]
        ]

    competition_results = utils.merge(
        (
            competition_results.reset_index(),
            competition_races[["eventId", "Date", "DisplayName"]],
            competition_events["boatClassId"],
            boat_classes,
        ),
        left_on=("raceId", "eventId", "boatClassId"),
        right_on="id",
    )

    results = competition_results.loc[
        competition_results.ResultTime > pd.to_timedelta(0),
        [
            "DisplayName_y", #comes from boat_classes and gives the code of the boat class
            "ResultTime",
            "Country",
            "Rank",
            "Date",
            "DisplayName_x" #comes from competition_races and refers to if it's Final A or B
        ],
    ]
    results.columns = ["Boat", "Time", "Country", "Rank", "Date", 'FinalType']
    results["Rank"] = results.Rank.fillna(0).astype(int)
    results.Time = results.Time.dt.total_seconds().apply(utils.format_totalseconds)
    return results.sort_values(by=['Date','Boat','FinalType', 'Rank']).reset_index(drop=True)

While running the next cell, I had to ammend some of 'api.py' in order to bypass errors without solving them. I'll get them sorted after all is loaded.

In [29]:
import itertools

def get_boat_codes(age = ["", "J", "B"], sex = ["M", "W"], boat_size = ["8+", "4+", "4-", "4x"]): 
    boat_codes = []
    list_products = itertools.product(age,sex,boat_size)
    for l in list_products:
        a,b,c=l
        boat_codes.append(a+b+c)
    return boat_codes

def get_boat_codes_df(boat_codes):
    b = api.get_boat_types()
    b = b.loc[b['DisplayName'].isin(boat_codes)]
    return b

boat_codes=get_boat_codes()
boat_codes_df = get_boat_codes_df(boat_codes)
results = pd.DataFrame()

for index,row in competitions_df.iterrows():
    df = get_competitions_results_REDO(competition_id=index, finals_only=True, boat_classes=boat_codes_df)
    print(row['Year'])
    
    if df.empty: continue
    df = df[['Boat', 'Time', 'Country', 'Rank','Date','FinalType']]
    df['Year'] = row['Year']
    df['City'] = row['City']
    results = pd.concat([results, df])

1980
1980
1981
1981
1982
1982
1983
1983
1984
1985
1985
1986
1986
1987
1987
1988
1989
1989
1990
1990
1991
1991
1992
1993
1993
1994
1994
1995
1995
1996
1997
1997
1998
1998
1999
1999
2000
2001
2001
2002
2002
2003
2003
2004
2005
2005
2005
2006
2006
2006
2007
2007
2007
2009
2009
2009
2010
2010
2010
2011
2011
2011
2012
2013
2013
2013
2014
2014
2014
2015
2015
2015
2016
2016
2016
2017
2017
2017
2018
2018
2018
2019
2019
2019
2021
2021
2022
2022
2022
2023
2023


ValueError: expected hh:mm:ss format before .

In [36]:
display(results[results['Year']==2023])

,Boat,Time,Country,Rank,Date,FinalType,Year,City
0,BM4-,6:09.38,ESP,1,2023-07-22 06:01:00,U23 Men's Four Final C,2023,Plovdiv
1,BM4-,6:20.64,EGY,2,2023-07-22 06:01:00,U23 Men's Four Final C,2023,Plovdiv
2,BM4x,5:53.87,NZL,1,2023-07-22 06:09:00,U23 Men's Quadruple Sculls Final C,2023,Plovdiv
3,BM4x,5:54.42,CRO,2,2023-07-22 06:09:00,U23 Men's Quadruple Sculls Final C,2023,Plovdiv
4,BM4x,6:03.82,CHI,3,2023-07-22 06:09:00,U23 Men's Quadruple Sculls Final C,2023,Plovdiv
...,...,...,...,...,...,...,...,...
68,JM4x,6:16.85,ITA,2,2023-08-05 11:31:00,U19 Men's Quadruple Sculls Final B,2023,Paris
69,JM4x,6:17.54,FRA,3,2023-08-05 11:31:00,U19 Men's Quadruple Sculls Final B,2023,Paris
70,JM4x,6:19.16,ROU,4,2023-08-05 11:31:00,U19 Men's Quadruple Sculls Final B,2023,Paris
71,JM4x,6:19.63,NZL,5,2023-08-05 11:31:00,U19 Men's Quadruple Sculls Final B,2023,Paris


1. Some ranks are missing
1. Some of the datetimes were missing and are noted down as 0001-01-01 00:00:00, which is beyond what python's datetime can handle. We simply don't parse the dates and leave them in string format

In [40]:
display(results[results['Rank']==0])
display(results[results['Date']=="0001-01-01 00:00:00"].value_counts('Year'))
display(results[results['Date']=="1979-01-01 00:00:00"])

,Boat,Time,Country,Rank,Date,FinalType,Year,City
85,W4x,6:56.12,AUS,0,0001-01-01 00:00:00,Women's Quadruple Sculls Final FC,1995,Tampere
86,W4x,7:03.15,NOR,0,0001-01-01 00:00:00,Women's Quadruple Sculls Final FC,1995,Tampere


Year
1995    102
1985     69
2011      6
Name: count, dtype: int64

,Boat,Time,Country,Rank,Date,FinalType,Year,City


In [41]:
results.to_csv('world_rowing_results.csv')